# Shows how to access Google's Gemini LLM

## Prerequisites

1. You need an API key to access the Gemini API.
    - See the documentation, "Get a Gemini API key", at: https://ai.google.dev/gemini-api/docs/api-key
2. Choose a Gemini model to use.
    - See the documentation, "Gemini models", at: https://ai.google.dev/gemini-api/docs/models/gemini
    - They are free (as of 2024-12-29), but they have different rate limits,
        token limits, and supported data types.
3. Choose an API.
    - `google-generativeai`
        - We use this API in this notebook.
        - https://pypi.org/project/google-generativeai/
        - https://ai.google.dev/gemini-api/docs/quickstart?lang=python
        - This API provides more fine-grained control over model behavior, so
            you have greater ability to customize the LLM usage.
    - `google-genai`
        - We are _not_ using this API in this notebook.
        - https://pypi.org/project/google-genai/
        - https://googleapis.github.io/python-genai/
        - This API is more concise and simplified.
        - The documentation says, "Please do not use this SDK in production."

In [1]:
import dotenv
import os
import google.generativeai as generativeai
import json

In [2]:
# List of possible models: https://ai.google.dev/gemini-api/docs/models/gemini
GEMINI_MODEL_NAME = "gemini-2.0-flash-exp"

# The name of the key that you have defined as one of:
#   - an environment variable;
#   - in a .env file; or
#   - as a Secret in Google Colab.
GEMINI_API_KEY_NAME = "GEMINI_API_KEY"

## Load API Key

In [3]:
# Loads variables defined in '.env' file as environment variables.
#   override=False means that the value of the variable in the environment
#     takes precedence over the value of the variable in the '.env' file.
_ = dotenv.load_dotenv(override=False)
GEMINI_API_KEY_VALUE = os.environ.get(GEMINI_API_KEY_NAME)

In [4]:
# In Google Colab, you have two options:
# (1) Prompt the user for the key value:
# import getpass
# GEMINI_API_KEY_VALUE = getpass.getpass()
# -or-
# (2) Save your API Key in the Secrets, then read that value:
# from google.colab import userdata
# GEMINI_API_KEY_VALUE = userdata.get('GEMINI_API_KEY_NAME')

## Get a model that returns JSON

See:

- https://ai.google.dev/gemini-api/docs/quickstart?lang=python#make-first-request
- https://github.com/google-gemini/cookbook/blob/main/quickstarts/JSON_mode.ipynb
- https://cloud.google.com/vertex-ai/generative-ai/docs/model-reference/inference#python
- https://cloud.google.com/vertex-ai/generative-ai/docs/reference/python/latest/vertexai.generative_models.GenerationConfig

In [5]:
generativeai.configure(api_key=GEMINI_API_KEY_VALUE)
generation_configuration = generativeai.GenerationConfig(
    temperature=1.0,
    candidate_count=1,
    max_output_tokens=1024,
    response_mime_type="application/json",
)
model = generativeai.GenerativeModel(
    GEMINI_MODEL_NAME, generation_config=generation_configuration
)

## Submit a prompt

In [6]:
prompt = """
You are a creative writer.
I want you to provide a few paragraphs describing a city from the United States in the 1920s,
using a detective noir style.
The city has a corrupt government.
Gangsters on the South Side are making a lot of money
from bootleg liquor during Prohibition.
Detective Harston Cooper is trying to help Mrs. Lucille Robinson
to find her missing husband, Roland Robinson.
Provide the response as JSON in this format:
{
  "city": {
    "name": ,
    "description": ,
    "atmosphere": ,
    "mood":
  }
}
"""

In [7]:
raw_response = model.generate_content(prompt)
raw_response

response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=protos.GenerateContentResponse({
      "candidates": [
        {
          "content": {
            "parts": [
              {
                "text": "{\n  \"city\": {\n    \"name\": \"Steelhaven\",\n    \"description\": \"Steelhaven, a city choked by its own ambition, where skyscrapers clawed at the sky, casting long, ominous shadows over the grimy streets below. The air hung thick with the smell of coal smoke and desperation, a testament to the city's industrial heart. But beneath the veneer of progress, a rot had taken hold. The city's government, a greasy machine of graft and backroom deals, operated with impunity, leaving the honest folk to fend for themselves. Every cop on the force seemed to have a price, and justice was a commodity sold to the highest bidder. The South Side, a labyrinth of dimly lit alleys and speakeasies, was where the real power resided. Gangsters, flush with cash from the illega

In [8]:
response = json.loads(raw_response.text)
response

{'city': {'name': 'Steelhaven',
  'description': "Steelhaven, a city choked by its own ambition, where skyscrapers clawed at the sky, casting long, ominous shadows over the grimy streets below. The air hung thick with the smell of coal smoke and desperation, a testament to the city's industrial heart. But beneath the veneer of progress, a rot had taken hold. The city's government, a greasy machine of graft and backroom deals, operated with impunity, leaving the honest folk to fend for themselves. Every cop on the force seemed to have a price, and justice was a commodity sold to the highest bidder. The South Side, a labyrinth of dimly lit alleys and speakeasies, was where the real power resided. Gangsters, flush with cash from the illegal liquor trade, ruled the roost with an iron fist, their influence reaching every corner of Steelhaven.",
  'atmosphere': "A palpable tension hung in the air, a feeling that at any moment, the city might explode. The constant hum of the elevated train, t

In [9]:
response["city"]["name"]

'Steelhaven'

In [10]:
response["city"]["description"]

"Steelhaven, a city choked by its own ambition, where skyscrapers clawed at the sky, casting long, ominous shadows over the grimy streets below. The air hung thick with the smell of coal smoke and desperation, a testament to the city's industrial heart. But beneath the veneer of progress, a rot had taken hold. The city's government, a greasy machine of graft and backroom deals, operated with impunity, leaving the honest folk to fend for themselves. Every cop on the force seemed to have a price, and justice was a commodity sold to the highest bidder. The South Side, a labyrinth of dimly lit alleys and speakeasies, was where the real power resided. Gangsters, flush with cash from the illegal liquor trade, ruled the roost with an iron fist, their influence reaching every corner of Steelhaven."